In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

from torchvision.transforms import transforms
from datasets.cifar10 import CIFAR10Loader
from utils.io_utils import *
from torch.utils.tensorboard import SummaryWriter
import subprocess
import torch
from torch import optim
from models.network.DDN import DDN
from tqdm import tqdm

D:\software\anaconda\envs\torch2.0.1\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
DEVICE = 'cuda'
EPOCHS = 100
BATCH_SIZE = 10
EXP_NAME = "train"
CLS = 10
INPUT_SHAPE = (3,64,64)
BETA = 1e-2
GAMMA = 1e-4
ETA = 1e-3

In [3]:
pipeline = transforms.Compose([transforms.ToTensor(),
                        transforms.Resize(INPUT_SHAPE[1:], antialias=True),
                        transforms.Normalize(mean=(0.4914,0.4822,0.4465), std=(0.2023,0.1994,0.2010))
                        ])


root = 'D:/project/Discardable-Distributed-Networks/data'
dataLoader = CIFAR10Loader(root=root,transform=pipeline,batch_size=BATCH_SIZE)
model_path = f'D:/project/Discardable-Distributed-Networks/save/{EXP_NAME}/'
create_directory_if_not_exists(model_path)

log_path = f'D:/project/Discardable-Distributed-Networks/logs/tensorboard/{EXP_NAME}/'
create_directory_if_not_exists(log_path)
fileList = getFileList(log_path)
for file in fileList:
    os.remove(os.path.join(log_path,file))
writer = SummaryWriter(log_dir=log_path)

process = subprocess.Popen(f'tensorboard --logdir={log_path} --port=6677 --bind_all', shell=True)
# debug_print(f"open tensorboard, cmd: tensorboard --logdir={log_path}")
print(f"open tensorboard: 127.0.0.1:6677")

input_sample = torch.randn((1,) + INPUT_SHAPE).to(DEVICE)
print("input size",input_sample.shape)

Files already downloaded and verified
Files already downloaded and verified
open tensorboard: 127.0.0.1:6677
input size torch.Size([1, 3, 64, 64])


In [4]:
def train_Model(model, dataloader, optimizer, epoch,writer,model_name):
    model.train()

    total_loss = 0.0

    loop = tqdm(enumerate(dataloader) ,total=len(dataloader),position=0)
    for batch_index, (data,target) in loop:
        data, target = data.to(DEVICE), target.to(DEVICE)

        optimizer.zero_grad()

        logits = model(data)

        cls_loss = torch.nn.functional.cross_entropy(logits, torch.nn.functional.one_hot(target, num_classes=CLS).float())

        infoMin_loss = 0.0
        infoMax_loss = 0.0

        loss = cls_loss

        loss += model.infoMin_loss * BETA + model.infoMax_loss * GAMMA

        total_loss += cls_loss.item() + infoMin_loss + infoMax_loss

        loss.backward()
        optimizer.step()
        loop.set_description(f'Train Epoch [{epoch}/{EPOCHS}]')
        loop.set_postfix(loss = loss.item())
        # if batch_index % 3000 == 0:
        #     print("Train Epoch: {} \t Loss:{:.6f} \n ".format(epoch,loss.item()))

    avg_total_loss = total_loss / len(dataloader)
    avg_cls_loss = cls_loss.item() / len(dataloader)
    avg_infoMin_loss = infoMin_loss / len(dataloader)
    avg_infoMax_loss = infoMax_loss / len(dataloader)


    writer.add_scalar(f'Train/Loss/{model_name}/total', avg_total_loss, epoch)
    writer.add_scalar(f'Train/Loss/{model_name}/cls', avg_cls_loss, epoch)
    writer.add_scalar(f'Train/Loss/{model_name}/infoMin', avg_infoMin_loss, epoch)
    writer.add_scalar(f'Train/Loss/{model_name}/infoMax', avg_infoMax_loss, epoch)

def test_Model(model, dataloader,epoch, writer, maxAcc,model_path,model_name):
    model.eval()

    total_loss1 = 0.0
    total_correct1 = 0.0
    total_loss2 = 0.0
    total_correct2 = 0.0

    with torch.no_grad():
        loop = tqdm(enumerate(dataloader) ,total=len(dataloader),position=0)
        input_sample = None
        for batch_index, (data,target) in loop:
            data, target = data.to(DEVICE), target.to(DEVICE)
            input_sample = data
            logits1 = model.drop_forward(data,[0.0,0.0,0.0])
            logits2 = model.drop_forward(data,[0.0,0.3,0.6])

            cls_loss1 = torch.nn.functional.cross_entropy(logits1, torch.nn.functional.one_hot(target, num_classes=CLS).float())
            cls_loss2 = torch.nn.functional.cross_entropy(logits2, torch.nn.functional.one_hot(target, num_classes=CLS).float())

            loss1 = cls_loss1
            loss2 = cls_loss2

            total_loss1 += cls_loss1.item()
            total_loss2 += cls_loss2.item()

            pred1 = logits1.argmax(dim=1)
            correct1 = pred1.eq(target.view_as(pred1)).sum().item()
            total_correct1 += correct1
            pred2 = logits2.argmax(dim=1)
            correct2 = pred2.eq(target.view_as(pred2)).sum().item()
            total_correct2 += correct2

            total_loss1 += loss1.item()
            total_loss2 += loss2.item()

            loop.set_description(f'Test Epoch [{epoch}/{EPOCHS}]')
            loop.set_postfix(ACC1 = 100.0 * correct1 / target.size(0),ACC2 = 100.0 * correct2 / target.size(0))

        avg_loss1 = total_loss1 / len(dataloader)
        avg_loss2 = total_loss2 / len(dataloader)
        Accuracy1 = 100.0 * total_correct1 / len(dataloader.dataset)
        Accuracy2 = 100.0 * total_correct2 / len(dataloader.dataset)

        print("Test_Average loss1: {:4f},ACC1: {:4f} loss2: {:4f},ACC2: {:4f}\n".format(avg_loss1, Accuracy1,avg_loss2, Accuracy2,))
        # Add scalars to TensorBoard
        writer.add_scalar(f'Test/Loss1/{model_name}', avg_loss1, epoch)
        writer.add_scalar(f'Test/Accuracy1/{model_name}', Accuracy1, epoch)
        writer.add_scalar(f'Test/Loss2/{model_name}', avg_loss2, epoch)
        writer.add_scalar(f'Test/Accuracy2/{model_name}', Accuracy2, epoch)

        if maxAcc < Accuracy2:
            model_name = os.path.join(model_path,f'{model_name}.ckpt')
            torch.save(model.state_dict(), model_name)
            model.save_onnx_model(input_sample=input_sample,dir_path=os.path.join(model_path,"onnx"))
            print(f'save onnx and pth to {model_path}')
            return Accuracy2



        return maxAcc

In [5]:
def exp(drops, MODEL_NAME='DDN'):
    model = DDN(in_places=3, dropout_probs=drops,groups=3)
    model = model.to(DEVICE)

    optimizer= optim.Adam(model.parameters())

    print(f"train {MODEL_NAME}")
    print("------------------")

    Acc = 0.0
    for epoch in range(1, EPOCHS + 1):
        # train_Model(model, dataLoader.train_loader, optimizer, epoch,writer,MODEL_NAME)
        Acc=test_Model(model, dataLoader.test_loader,epoch,writer,Acc,model_path,MODEL_NAME)


In [ ]:
drops = [0,0.3,0.6]
exp(drops)


train DDN
------------------


Test Epoch [1/100]:  81%|████████  | 808/1000 [03:10<00:47,  4.03it/s, ACC1=0, ACC2=0]  